In [82]:
import pandas as pd 
import numpy as np
import math
import ReciprocalRank

In [33]:
pd.set_option('display.max_columns', None)

In [5]:
train = pd.read_csv('train.csv')
meta = pd.read_csv('item_metadata.csv')

In [12]:
tempt.head(10)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
9,00RL8Z82B2Z1,aff3928535f48,1541037542,10,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


# 1. train.csv 
1. `user_id` `session_id` : session의 의미는?
2. `timestamp` `step` : 시간 변수 (clickout이 몇번째 step에서 나왔나?)
3. `action_type` `reference` : 고객 행동 변수
4. `platform` `city` : 지역 변수
5. `device`
6. `current_filters` : `action_type`에서 'filter selection'을 한 user들의 누적 필터 ~ 고객 행동 변수
7. `impressions` `prices` : display된 아이템 , 가격 [item 당 가격 변동?]

### 1-1 `current filter`

In [35]:
# current filter 가 NA가 아닌 경우는 어떤 경우?
train[train.current_filters.notna()]

# `action_type` ==  filter selection,  `action_type` ==  clickout item 인 경우. -> 어차피 clickout item인 row에서 누적되서 나타남. 

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
125,03P4VFKK12UO,325fafb5fa450,1541107336,3,filter selection,Very Good Rating,US,"Bakersfield, USA",desktop,Very Good Rating,NaN,NaN
126,03P4VFKK12UO,325fafb5fa450,1541107366,4,filter selection,Hostal (ES),US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN
127,03P4VFKK12UO,325fafb5fa450,1541107366,5,filter selection,Hotel,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN
128,03P4VFKK12UO,325fafb5fa450,1541107366,6,filter selection,Motel,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN
129,03P4VFKK12UO,325fafb5fa450,1541107366,7,filter selection,Resort,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN
130,03P4VFKK12UO,325fafb5fa450,1541107366,8,filter selection,4 Star,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN
131,03P4VFKK12UO,325fafb5fa450,1541107366,9,filter selection,5 Star,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN
132,03P4VFKK12UO,325fafb5fa450,1541107368,10,filter selection,Hostal (ES),US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN
133,03P4VFKK12UO,325fafb5fa450,1541107368,11,filter selection,Hotel,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN
134,03P4VFKK12UO,325fafb5fa450,1541107368,12,filter selection,Motel,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN


In [36]:
print(train[train.current_filters.notna()].loc[train[train.current_filters.notna()].action_type == 'clickout item'].shape)
print("filter가 적용된 상태에서 clickout을 한 구매 개수는 383,500개")


(383500, 12)
filter가 적용된 상태에서 clickout을 한 구매 개수는 383,500개


* example 1: current_filters 여러번 있는 유저 

In [45]:
user001 = train.loc[train.user_id == '0O1CTEP95YKQ']

In [46]:
user001

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
469,0O1CTEP95YKQ,af793494b5f84,1541113259,1,change of sort order,interaction sort button,CO,"Las Vegas, USA",mobile,NaN,NaN,NaN
470,0O1CTEP95YKQ,af793494b5f84,1541113276,2,filter selection,Hotel,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN
471,0O1CTEP95YKQ,af793494b5f84,1541113276,3,filter selection,Motel,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN
472,0O1CTEP95YKQ,af793494b5f84,1541113276,4,filter selection,Resort,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN
473,0O1CTEP95YKQ,af793494b5f84,1541113314,5,filter selection,Breakfast Included,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort|Breakfast Included,NaN,NaN
474,0O1CTEP95YKQ,af793494b5f84,1541113324,6,filter selection,Breakfast Included,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN
475,0O1CTEP95YKQ,af793494b5f84,1541113325,7,filter selection,Resort,CO,"Las Vegas, USA",mobile,Hotel|Motel,NaN,NaN
476,0O1CTEP95YKQ,af793494b5f84,1541113325,8,filter selection,Motel,CO,"Las Vegas, USA",mobile,Hotel,NaN,NaN
477,0O1CTEP95YKQ,af793494b5f84,1541113329,9,filter selection,Hotel,CO,"Las Vegas, USA",mobile,NaN,NaN,NaN
478,0O1CTEP95YKQ,af793494b5f84,1541113330,10,filter selection,Breakfast Included,CO,"Las Vegas, USA",mobile,Breakfast Included,NaN,NaN


1.  current filter는 'action type'이 clickout item일때만 보면 된다. (누적됨)
2.  한 session에서 clickout이 여러번 나올 수 있기 때문에 마지막 step이 아닌 step에서 clickout이 나타날 수 있다. ex. step 1 - clickout, step 2 - clickout ... 
3.  clickout item일 때 refernece가 같은 item이 연달아 (연이은 step) 나오는 경우는 뭐지? 두 번 산건가? 오류인건가? 

     -> answered: timestamp가 초 단위라 step 컬럼을 도입했다는데 동문서답 오졌다 ! 
     
     -> my opinion: delay 때문에 더블 클릭 한 경우 

### 1-2 `action_type`

In [38]:
train.action_type.value_counts()

interaction item image     11860750
clickout item               1586586
filter selection             695917
search for destination       403066
change of sort order         400584
interaction item info        285402
interaction item rating      217246
interaction item deals       193794
search for item              152203
search for poi               137444
Name: action_type, dtype: int64

In [40]:
train.loc[train.action_type == 'change of sort order'].reference.value_counts()

interaction sort button     235027
price only                   78863
price and recommended        43317
distance only                12413
distance and recommended     10102
rating and recommended        9833
rating only                   7159
our recommendations           3870
Name: reference, dtype: int64

In [41]:
train.loc[train.action_type == 'filter selection'].reference.value_counts()

Sort by Price                      78922
Hotel                              58039
5 Star                             46193
Best Value                         43319
4 Star                             42625
Resort                             42343
Hostal (ES)                        35028
Motel                              33698
Focus on Distance                  32605
3 Star                             24335
Free WiFi (Combined)               20848
Very Good Rating                   20351
Breakfast Included                 19612
Excellent Rating                   18801
House / Apartment                  18720
Swimming Pool (Combined Filter)    14975
Car Park                           14203
Good Rating                        12687
Sort By Distance                   12423
Serviced Apartment                 11406
Focus on Rating                     9832
Air Conditioning                    7885
Spa (Wellness Facility)             7872
Sort By Rating                      7159
2 Star          

`change of sort order` : price, distance, reccomendation filter

`filter selection` : more filters(free wifi, ...), 'sort by price'?

1. `filter selection`의 reference에 'sort by price'와 'focus on distnace', 'sort by distance', 'sort by popularity'등이 있다. 이것들은 무슨 필터를 사용한 것일까? 

`interaction item deals ` : 'view deal'버튼을 클릭한 것인데 `clickout item`과 같은 행동이지 않아?


### 1-3 `timestamp`

In [47]:
import datetime
def myfunc(x):
    return datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')

In [50]:
datelist = list(map(myfunc,list(user001.timestamp)))
user001['datetime'] = datelist

In [62]:
cols = user001.columns.tolist()

In [69]:
cols = cols[-1:] + cols[:-1]

In [70]:
user001 = user001[cols]

In [71]:
user001

,datetime,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
469,2018-11-01 23:00:59,0O1CTEP95YKQ,af793494b5f84,1541113259,1,change of sort order,interaction sort button,CO,"Las Vegas, USA",mobile,NaN,NaN,NaN
470,2018-11-01 23:01:16,0O1CTEP95YKQ,af793494b5f84,1541113276,2,filter selection,Hotel,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN
471,2018-11-01 23:01:16,0O1CTEP95YKQ,af793494b5f84,1541113276,3,filter selection,Motel,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN
472,2018-11-01 23:01:16,0O1CTEP95YKQ,af793494b5f84,1541113276,4,filter selection,Resort,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN
473,2018-11-01 23:01:54,0O1CTEP95YKQ,af793494b5f84,1541113314,5,filter selection,Breakfast Included,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort|Breakfast Included,NaN,NaN
474,2018-11-01 23:02:04,0O1CTEP95YKQ,af793494b5f84,1541113324,6,filter selection,Breakfast Included,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN
475,2018-11-01 23:02:05,0O1CTEP95YKQ,af793494b5f84,1541113325,7,filter selection,Resort,CO,"Las Vegas, USA",mobile,Hotel|Motel,NaN,NaN
476,2018-11-01 23:02:05,0O1CTEP95YKQ,af793494b5f84,1541113325,8,filter selection,Motel,CO,"Las Vegas, USA",mobile,Hotel,NaN,NaN
477,2018-11-01 23:02:09,0O1CTEP95YKQ,af793494b5f84,1541113329,9,filter selection,Hotel,CO,"Las Vegas, USA",mobile,NaN,NaN,NaN
478,2018-11-01 23:02:10,0O1CTEP95YKQ,af793494b5f84,1541113330,10,filter selection,Breakfast Included,CO,"Las Vegas, USA",mobile,Breakfast Included,NaN,NaN


1. timestamp 간격이 넓은 구간에서 session id가 바뀜. 
2. 11월 1일 8시에 접속해서 clickout 하고 (뒤에 나옴) 11월 1일 11시에 접속해서 filtering한다음 clickout한다. 즉, timestamp가 ordering 되있지 않다는 의미인가?! 

### 1-4 실제 user들은 `impression`에서 어떤 것을 `clickout` 했을까

In [73]:
def string_to_array(s):
    """convert pipe separated string to array"""
    
    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string or nan")
    
    return out  

In [92]:
user001.loc[user001.action_type == 'clickout item']

,datetime,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
487,2018-11-01 23:04:34,0O1CTEP95YKQ,af793494b5f84,1541113474,19,clickout item,63051,CO,"Las Vegas, USA",mobile,Breakfast Included|Sort by Price,69086|8533|925367|58949|61049|8581|8517|77520|...,59|66|78|86|87|90|90|92|93|98|99|100|106|108|1...
488,2018-11-01 23:05:09,0O1CTEP95YKQ,af793494b5f84,1541113509,20,clickout item,2851635,CO,"Las Vegas, USA",mobile,Breakfast Included|Sort by Price,69086|8533|925367|58949|61049|8581|8517|77520|...,59|66|78|86|87|90|90|92|93|98|99|100|106|108|1...
490,2018-11-01 23:05:37,0O1CTEP95YKQ,af793494b5f84,1541113537,22,clickout item,8602,CO,"Las Vegas, USA",mobile,Breakfast Included|Sort by Price,69086|8533|925367|58949|61049|8581|8517|77520|...,59|66|78|86|87|90|90|92|93|98|99|100|106|108|1...
3752350,2018-11-01 20:27:47,0O1CTEP95YKQ,64fd485f2ca59,1541104067,1,clickout item,8595,CO,"Las Vegas, USA",mobile,NaN,8551|8589|57049|8613|8586|8561|8564|8577|8607|...,411|168|131|102|152|100|143|107|80|160|395|350...
3752351,2018-11-01 20:28:44,0O1CTEP95YKQ,64fd485f2ca59,1541104124,2,clickout item,8595,CO,"Las Vegas, USA",mobile,NaN,8551|8589|57049|8613|8586|8561|8564|8577|8607|...,411|168|131|102|152|100|143|107|80|160|395|350...


In [80]:
user001_impression_list = string_to_array(user001.loc[user001.action_type == 'clickout item'].iloc[0,-2])

In [81]:
user001_impression_list

['69086',
 '8533',
 '925367',
 '58949',
 '61049',
 '8581',
 '8517',
 '77520',
 '8531',
 '4169220',
 '63051',
 '397316',
 '840726',
 '2851635',
 '1493341',
 '8804492',
 '62454',
 '450176',
 '102082',
 '148954',
 '8602',
 '441851',
 '63858',
 '8516',
 '8510248']

In [90]:
user001_rank = ReciprocalRank.ReciprocalRank(user001_impression_list,user001.loc[user001.action_type == 'clickout item'].iloc[0,6]).mrr()

In [91]:
# first clickout score
user001_rank

0.5666666666666667

In [93]:
user001_rank = ReciprocalRank.ReciprocalRank(user001_impression_list,user001.loc[user001.action_type == 'clickout item'].iloc[1,6]).mrr()

In [94]:
# second clickout score
user001_rank

0.4047619047619047

In [95]:
user001_rank = ReciprocalRank.ReciprocalRank(user001_impression_list,user001.loc[user001.action_type == 'clickout item'].iloc[2,6]).mrr()

In [96]:
# thrid clickout score
user001_rank

0.0625

* conclusion


1. filter가 'sort by price' 걸려서 impression이 price의 오름차순으로 정렬되있다. 그래서 상위에 있는 것 clickout 한 다음 밑에 있는 아이템 하나씩 click out 누른 것 같다.
2. 그럼, filter (price, distance, rating) 별로 다르게 ranking을 구성해야한다는 뜻 ?! `filter`가 매우 중요하다! -> FE! 

# 2. meta data.csv

In [7]:
meta.head()

,item_id,properties
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
2,5834,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
3,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone...
4,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Sa...


In [106]:
meta.properties[0]

'Satellite TV|Golf Course|Airport Shuttle|Cosmetic Mirror|Safe (Hotel)|Telephone|Hotel|Sitting Area (Rooms)|Reception (24/7)|Air Conditioning|Hypoallergenic Rooms|Cable TV|Hotel Bar|Pool Table|Bathtub|Satisfactory Rating|Room Service|Luxury Hotel|Terrace (Hotel)|Television|Minigolf|Business Hotel|Shower|Cot|Gym|Hairdryer|Hypoallergenic Bedding|Accessible Parking|From 3 Stars|Good Rating|Radio|4 Star|From 4 Stars|Family Friendly|Desk|Tennis Court (Indoor)|Balcony|WiFi (Public Areas)|Openable Windows|Express Check-In / Check-Out|Restaurant|Laundry Service|Ironing Board|Tennis Court|From 2 Stars|Business Centre|Bowling|Conference Rooms|Electric Kettle|Accessible Hotel|Porter|Bike Rental|Non-Smoking Rooms|Car Park|Safe (Rooms)|Fitness|Fan|Flatscreen TV|Computer with Internet|WiFi (Rooms)|Lift|Central Heating'

* idea for FE
1. number of properties 
2. properties를 group으로 나누기 (ex. accessibility -> airport shuttle / facility -> TV, bathtub / ...)
3. Filter == `rating only` 구분하는데 data를 줄 수 있음


### 2-1 df_item_v2.csv 

In [107]:
d2 = pd.read_csv('df_item_v2.csv')

In [108]:
d2.head(20)

,item_id,properties,mean_price,min_price,max_price,var_price,std_price,n_impressions,n_interaction_item_info,n_interaction_item_image,n_interaction_item_rating,n_interaction_item_deals,n_search_for_item,n_clickout_item
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...,124.191083,64.0,472.0,2505.504889,50.055019,157.0,NaN,6.0,NaN,1.0,NaN,7.0
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...,102.313433,59.0,270.0,765.991312,27.676548,134.0,1.0,35.0,NaN,1.0,NaN,6.0
2,5834,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...,93.161932,38.0,319.0,2334.334574,48.314952,352.0,NaN,89.0,2.0,NaN,NaN,17.0
3,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone...,153.838710,91.0,417.0,2178.030437,46.669374,248.0,NaN,108.0,NaN,1.0,2.0,7.0
4,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Sa...,93.651376,47.0,225.0,1776.392223,42.147268,109.0,NaN,64.0,NaN,1.0,1.0,13.0
5,6094,Satellite TV|Sailing|Safe (Hotel)|Telephone|Ho...,112.773585,44.0,204.0,738.967604,27.183959,53.0,NaN,NaN,NaN,NaN,NaN,3.0
6,6288,Satellite TV|Airport Shuttle|Cosmetic Mirror|S...,108.782609,59.0,175.0,698.315060,26.425652,69.0,NaN,2.0,1.0,1.0,NaN,NaN
7,6358,Cosmetic Mirror|Safe (Hotel)|Telephone|Hotel|C...,211.076023,60.0,640.0,26242.280770,161.994694,171.0,NaN,13.0,NaN,2.0,1.0,2.0
8,6456,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...,165.220339,65.0,540.0,4764.081394,69.022325,354.0,3.0,188.0,2.0,1.0,NaN,14.0
9,6561,Satellite TV|Golf Course|Cosmetic Mirror|Safe ...,112.723089,44.0,731.0,5364.643289,73.243725,1282.0,3.0,145.0,1.0,7.0,NaN,43.0


1. rating 추가
2. property group 추가
3. distance 정보 추가 (?) - distance 연구 더 해야할 듯